# Analiza danych tekstowych - text mining

In [ ]:
import nltk
from bs4 import BeautifulSoup
import numpy as np

## Obróbka tekstu

Najważniejszy element analizy tekstów!

In [ ]:
wiki = "http://en.wikipedia.org/wiki/"
titles = ["Integral", "Riemann_integral", "Riemann-Stieltjes_integral", "Derivative",
    "Limit_of_a_sequence", "Edvard_Munch", "Vincent_van_Gogh", "Jan_Matejko",
    "Lev_Tolstoj", "Franz_Kafka", "J._R._R._Tolkien"]

from urllib.request import urlopen

def parse(url):        
    x = urlopen(url)
    x = x.read()
    x = BeautifulSoup(x)
    x = x.find("div",id="bodyContent")
    x = x.find_all("p")
    return(x)

articles = [parse(url) for url in [wiki+x for x in titles]]

In [ ]:
articles[0]

In [ ]:
import re
articles = [re.sub("<.+?>"," ",str(a)) for a in articles]
articles[0]

In [ ]:
articles = [nltk.word_tokenize(a) for a in articles]
#articles = nltk.tokenize.regexp_tokenize(articles[0],"(\w+[-\w+]?)|\d+") # mozna samemu regexem okreslic deffinicje "slowa"
articles[0]

In [ ]:
?nltk.word_tokenize

In [ ]:
?nltk.TreebankWordTokenizer

In [ ]:
import string
articles = [[w for w in a if w not in string.punctuation] for a in articles]
articles[0]

In [ ]:
articles = [[w.lower() for w in a] for a in articles]
articles[0]

In [ ]:
articles = [[w for w in a if w not in nltk.corpus.stopwords.words("english")] for a in articles]
articles[0]

In [ ]:
stemmer = nltk.PorterStemmer()
articles = [[stemmer.stem(w) for w in a] for a in articles]
articles[0]

In [ ]:
articles[0]

In [ ]:
articles = [' '.join(x) for x in articles]

In [ ]:
articles[0]

# Reprezentacje tekstu

## Podstawowa reprezentacja macierzowa - liczności

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
?CountVectorizer

In [ ]:
c = CountVectorizer(token_pattern='(?u)\\b\\w+\\b',min_df=3,max_df=0.5)
dtm = c.fit(articles)
art = dtm.transform(articles)
art

In [ ]:
dtm.vocabulary_

In [ ]:
dtm.get_feature_names()

In [ ]:
art

In [ ]:
art.getcol(dtm.get_feature_names().index("kingdom")).todense()

In [ ]:
art.getrow(0).todense()

In [ ]:
words = np.array(dtm.get_feature_names())

## Zadanie - stworzyć macierz ze słowami, w której i-ty wiersz przechowuje k najczęstszych słów z i-tego dokumentu.

Mamy tu do czynienia z macierzami rzadkimi - wymagają specjalnej obsługi, bo w normalnej postaci byłyby za duże.

In [ ]:
def top_words(M,dtm,k):
    words = np.array(dtm.get_feature_names())
    return(np.array([words[np.squeeze(np.array(np.argsort(M[i,:].todense())))[-k:]] for i in range(M.shape[0])]))

top_words(art,dtm,6)

# LSI

Czyli SVD przy użyciu gotowego narzędzia

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=3) # CHCEMY WZIĄĆ Z MACIERZY V TYLKO 3 CEHCY UKRYTE
svd.fit(art)
lsi = svd.transform(art) # CZYLI TO JEST TA NASZA V!

lsi # PRZEANALIZUJMY PODOBIEŃSTWO - WIERSZE TO DOKUMENTY - JEST DOBRZE!

## TF-IDF

### https://en.wikipedia.org/wiki/Tf%E2%80%93idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
?TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
art = tfidf.fit_transform(articles)
art

In [ ]:
top_words(art,tfidf,6)

### Pytanie: czy tfidf można zastosować do czegoś innego niż wazności słów w teksście?

### Uwaga: do mierzenia podobieństwa tekstów dobrze sprawdza się miara cosinusowa! (np. w grupowaniu)

https://en.wikipedia.org/wiki/Cosine_similarity

Mierzy ona podobieństwo wektorów na podstawie rozkładu wartości elementów (proporcji), a nie wartości bewzględnych - matematycznie - podobieństwo jest określane na podstawie kąta pomiędzy wektorami (wartości cosinusa tego kąta), a nie na podstawie długości wektorów.

## Zadanie - otagować teksty korpusu 20newsgroups

In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

In [ ]:
from pprint import pprint
pprint(list(newsgroups_train.target_names)) # kategorie tematyczne:

In [ ]:
newsgroups_train.target # kategorie kolejnych dokumentów

In [ ]:
newsgroups_train.data[0]

# Model LDA - Latent Dirichlet Allocation (ukryta alokacja Dirichleta)


Motywacja: przedstawienie tekstu jako mieszanki tematów.


Temat - rozkład prawdopodobieństwa na zbiorze słów.


Przykład:
*  <s>Mam</s> gorączkę <s>i</s> katar.
* Graliśmy <s>i</s> siatkówkę.
* Sport <s>to</s> zdrowie.


Ile "tematów" widzimy?
<br>

<br>

<br>

<br>

<br>

<br>

Intyicyjnie: dwa tematy: "sport" oraz "zdrowie"
* Pierwsze zdanie = 100% zdrowie
* Drugie zdanie = 100% sport
* Trzecie zdanie = 50% sport + 50% zdrowie


## Rozkład Dirichleta 

Jest to rozkład, na którym opiera się model LDA

Gęstość trójwymiarowego rozkładu Dirichleta Dir($\alpha$).

Wektor losowy $(x_1 , ..., x_K )$ z $K$-wymiarowego rozkładu Dirichleta to punkt na $(K-1)$-wymiarowym sympleksie, czyli $x_1 + ... + x_K = 1$, $x_i \geq 0$.

$\alpha=3$ | $\alpha=0.95$
- | - 
![alt](dir3.jpg) | ![alt](dir095.jpg)



## Ale $\alpha$ może być wektorem, czyli $\alpha = (\alpha_1, ..., \alpha_K)$. Co wówczas?

<img src="dirichlet.png" widht="200">

Źródło: http://jonathan-huang.org/research/hln/hlnfit.html

### Czyli rozkład jest niesymetryczny. Odpowiada to temu, że tematy mogą mieć różną częstość, co bardzo odpowiada rzeczywistości. Wtedy wartość oczekiwana $X_i$ (i-tej współrzędnej wektora z rozkładu Dirichleta) wynosi 
## $$\frac{\alpha_i}{\sum\limits_{j=1}^K \alpha_j}$$

czyli średni udział $i$-tego tematu jest proporcjonalny do $\alpha_i$.

Wizualizacja wartości z rozkłau Dirichleta:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.figure(figsize=(15,4))
for i in range(1,13):
    plt.subplot(2,6,i)
    plt.bar(range(5),np.random.dirichlet((0.5, 0.1, 0.1, 0.1, 0.1), 1)[0])
    plt.ylim(0,1)
    plt.xticks(range(5))
plt.show()

# PIERWSZY JEST NAJCZĘSTSZY (bo piwerwsza wartość w wektorze alfa jest największa)

In [ ]:
s = np.random.dirichlet((0.2, 0.1, 0.1), 20).transpose()

plt.barh(range(20), s[0])
plt.barh(range(20), s[1], left=s[0], color='g')
plt.barh(range(20), s[2], left=s[0]+s[1], color='r')
plt.title("Lengths of Strings")
plt.show()

<br>

<br>

<br>

<br>

# Model LDA

<img src="LDA_doc.jpg">

<br>

<br>

<br>

<br>

<img src="Smoothed_LDA.png">


,gdzie

$\theta_d \sim Dir(\alpha)$  - rozkład tematów w dokumencie

$Z \sim Discr(\theta)$ - temat, którego pochodz słowo

$W \sim Discr(\phi_Z)$ - słowo

$\phi_i \sim Dir(\beta)$ - tematy

Dokumenty będą składać sie tylko z kilku tematów (alfa będzie mała)

Tematy będą charakteryzowane również tylko przez cześć słów (beta małe). Dzięki temu możemy ludzkim okiem rozróżnić i zintepretować tematy.

# Uwaga

W modelu LDA są 3 parametry: K - liczba tematów, wektor lub skalar alfa oraz wektor lub skalar beta.

Dobre oprogramowanie do pracy z modelem LDA to takie, które wymaga podania od użytkownika jedynie K, a parametry alfa i beta estymuje sobie sam (wykorzystując uznane metody estymacji optymalnych ich wartości) i pozwala wybrać czy mają być skalarami czy wektorami. Alfa w praktyce powinna być wektorem - róże wartości elementów wektora odpowiadają różnym częstościom tematów. Natomiast Beta czasem może dać lepsze wyniki jako wektor, a czasem jako skalar. Takim narzędziem jest np. MALLET (http://mallet.cs.umass.edu/). W modułach Pythona takich elastycznych funkcjonalności nie ma, ale można z tym żyć.

Istnieją dwie powszechnie stosowane metody dopasowaywania modelu: wnioskowanie wariacyjne oraz próbkowanie Gibbsa. Ta druga metoda jest lepsza. (https://stats.stackexchange.com/questions/8485/a-good-gibbs-sampling-tutorials-and-references)

### LDA w module GENSIM

gensim to modul stworzony do analizy tekstów: https://radimrehurek.com/gensim/

In [ ]:
from gensim import corpora, models
from gensim.models.ldamodel import LdaModel

a = [nltk.word_tokenize(x) for x in articles]

dictionary = corpora.Dictionary(a)
corpus = [dictionary.doc2bow(text) for text in a]

In [ ]:
dictionary.keys()[:10]  # - ID slow

In [ ]:
dictionary[0]   # - slowa

In [ ]:
corpus[0] # - dokumenty w postali listy krotek (id slowa, licznosc wystapien tego slowa)

In [ ]:
model = LdaModel(corpus=corpus,id2word=dictionary,num_topics=3,alpha="auto")

In [ ]:
?LdaModel

"auto" dla parametrów alfa i eta (eta jest uzywane zamiennie z beta) oznacza, że model znajdzie optymalne wartości, ale będą to wektory.

Do uczenia modelu wykorzystywane jest wnioskowanie wariacyjne.

### Wypisania rozkładów tematów w dokumentach (jest trudne bo dane są zapisane w formacie rzadkim zapisana jest tylko cześć tematów - te, które mają bardzo niski udział zostają pominięte)

In [ ]:
for i in range(len(articles)):
    print(model.get_document_topics(corpus[i],minimum_probability=0.1))

Rozkład słów w temacie $k$ - analogicznie jak wyżej, dane są w formacie rzadkim, dlatego mamy tylko najczęstsze słowa. Są to krotki (id_slowa, prawdopodobieństwo wystapienia słowa w tym temacie)

In [ ]:
k = 0
model.get_topic_terms(topicid=k)

# LDA w module lda

In [ ]:
import lda
import lda.datasets

In [ ]:
# Wczytujemy dane: jest to zbiór notek prasowych agencji REUTERS

X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
print(X)                                # - macierz wystapien slow
print(vocab[:10])                       # - lista slow
print(titles[:10])                      # - tytuly dokumentow

In [ ]:
?lda.LDA

Ten moduł działa dla ustalonych parametrów alfa i beta - to jest słabe.

Ten moduł dopasowuje model przy użyciu próbkowania Gibbsa - to jest dobre (n_iter to parametr tej metody - liczba iteracji)

In [ ]:
model = lda.LDA(n_topics=20, n_iter=500,random_state=1)
model.fit(X)

#Funkcja drukuje postęp uczenia modelu - iteracje i wartość wunkcji log wiarogodności (i na początku kilka dodatkowych info)

In [ ]:
topic_word = model.topic_word_
n_top_words = 8

print(topic_word) # macierz prawdopodobieństw słów w tematach

In [ ]:
# wypisanie najczestszych slow w tematach
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

In [ ]:
model.components_ # tematy - jeden wiersz = jeden temat   to samo model.topic_word_

In [ ]:
model.loglikelihood() # wzrtość funkcji log wiarogodności modelu 

### Zadanie: Pogrupować zbiór 20NEWSGROUP.

Przetestować różne reprezentacje korpusu:

- macierz liczności wystąpień słów
- macierz tfidf
- lsi
- lda

# Materiały i źródła:

https://radimrehurek.com/gensim/wiki.html

http://radimrehurek.com/topic_modeling_tutorial/2%20-%20Topic%20Modeling.html

https://www.ariddell.org/lda.html

pokazać na koniec modelu LDA: http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb

http://brandonrose.org/clustering

http://totoharyanto.staff.ipb.ac.id/files/2012/10/Building-Machine-Learning-Systems-with-Python-Richert-Coelho.pdf

https://de.dariah.eu/tatom/topic_model_python.html